In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import string
import nltk
nltk.download('stopwords')
import annoy
import numpy as np
import tensorflow as tf
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm import tqdm_notebook
from transformers import TFAutoModel, AutoTokenizer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [ ]:
def preprocess_txt(line):
    line=re.sub(re.compile(r'[^\w\s]'), " ", line)
    line=re.sub(re.compile(r'[^a-zA-Zа-яА-Я0-9]'), " ", line)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    #spls = [i for i in spls if i != ""]
    return ' '.join(spls)

In [ ]:
question = []
answer = []
errors = []
with open("/content/drive/MyDrive/Colab Notebooks/CHAT_BOT/planeta_qwestions.txt", "r") as f:
    for line in tqdm_notebook(f):
        spls = line.split("\t")
        if len(spls)==1:
            errors.append(spls)
            continue
        question.append(spls[0].strip())
        answer.append(spls[1].strip())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


0it [00:00, ?it/s]

In [ ]:
#errors

In [ ]:
planeta_qwestions = pd.DataFrame({'qwest': question, 'ans': answer})

In [ ]:
planeta_qwestions

,qwest,ans
0,Вы продаете дачу?,"да, продаю."
1,Вы продаете участок?,"да, продаю."
2,Вы продаете коттедж?,"да, продаю."
3,Сколько стоит?,1 тугрик.
4,Какая стоимость?,1 тугрик.
...,...,...
360,Председатель правления,Рoмaнoв Рoмaн Алeкcaндрoвич.
361,Члены правления,Виниченко Иван Михайлович; Маркелов Андрей Вла...
362,Режим работы правления,Правление собирается каждую субботу с 10 до 14...
363,Расписание работы правления,Правление собирается каждую субботу с 10 до 14...


In [ ]:
%%time

with Pool(3) as p:
    qwest = list(tqdm(p.imap(preprocess_txt, planeta_qwestions['qwest'])))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


0it [00:00, ?it/s]

CPU times: user 109 ms, sys: 54 ms, total: 163 ms
Wall time: 709 ms


In [ ]:
planeta_qwestions['morpher_qwest'] = qwest

In [ ]:
planeta_qwestions.head()

,qwest,ans,morpher_qwest
0,Вы продаете дачу?,"да, продаю.",продавать дача
1,Вы продаете участок?,"да, продаю.",продавать участок
2,Вы продаете коттедж?,"да, продаю.",продавать коттедж
3,Сколько стоит?,1 тугрик.,стоить
4,Какая стоимость?,1 тугрик.,стоимость


In [ ]:
morpher_qwest_tokens = []
for i in planeta_qwestions.morpher_qwest:
    tokens = i.split()
    if len(tokens) < 2:
        continue
    morpher_qwest_tokens.extend(tokens)

In [ ]:
planeta_qwestions

,qwest,ans,morpher_qwest
0,Вы продаете дачу?,"да, продаю.",продавать дача
1,Вы продаете участок?,"да, продаю.",продавать участок
2,Вы продаете коттедж?,"да, продаю.",продавать коттедж
3,Сколько стоит?,1 тугрик.,стоить
4,Какая стоимость?,1 тугрик.,стоимость
...,...,...,...
360,Председатель правления,Рoмaнoв Рoмaн Алeкcaндрoвич.,председатель правление
361,Члены правления,Виниченко Иван Михайлович; Маркелов Андрей Вла...,член правление
362,Режим работы правления,Правление собирается каждую субботу с 10 до 14...,режим работа правление
363,Расписание работы правления,Правление собирается каждую субботу с 10 до 14...,расписание работа правление


In [ ]:
#morpher_qwest_tokens

In [ ]:
bert = TFAutoModel.from_pretrained("setu4993/LaBSE")
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at setu4993/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
%%time

bert_index = annoy.AnnoyIndex(768 , 'angular')

index_map = {}
counter = 0
for i in range(len(planeta_qwestions)):
    index_map[counter] = planeta_qwestions.loc[i]['ans']
    question = planeta_qwestions.loc[i]['morpher_qwest']
        
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    out_state = bert(**tok)[1].numpy()[0]
    bert_index.add_item(counter, out_state)
    counter += 1
        

bert_index.build(10)

CPU times: user 1min 45s, sys: 4.17 s, total: 1min 49s
Wall time: 1min 33s


In [ ]:
bert_index.save('bert_index_planeta')

True

In [ ]:
bert_index = annoy.AnnoyIndex(768, 'angular')
bert_index.load('bert_index_planeta') 

True

In [ ]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    vector = bert(**tok)[1].numpy()[0]
    answers = index.get_nns_by_vector(vector, 1, )
    return [index_map[i] for i in answers]

In [ ]:
get_response("Из чего сделана баня?", bert_index, index_map)

['Дом-баня размером 6 х 8 м и пристройкой 2 х 8 м имеет теплоизолированный фундамент на автомобильных покрышках, заполненных глиной исключающих появление конденсата на нижней обвязке дома из лиственничного бруса 20 х 20 см. Высота потолка – 2,6 м. На 1,80 м от стен отмостка утеплена экструдированным пенополистиролом сверху обтянутым георешеткой, исключающим промерзание подпола в случае зимнего отопления. Летом грунт под домом в июле еще не прогрет, что создает прохладу на первом этаже дома. Каркасные стены дома сделаны из сосновой доски 15 х 4 см собранных на гвоздях и усиленных металлическими уголками на мебельных конфирматах (самый мощный мебельный крепеж). Снаружи по диагонали стены обшиты лиственничной имитацией бруса. Изнутри кедровой и лиственничной имитацией бруса. Внутри стен и на потолке эковата. Пол комнаты из лиственничной половой рейки. Пол парилки и мойки облицован керамогранитом. Банная печь «Сударушка». Топка чугунная, каменка из нержавейки, под облицовку кирпичом, с бол

In [ ]:
get_response("сколько стоит?", bert_index, index_map)

['1 тугрик.']

In [ ]:
get_response("как добраться?", bert_index, index_map)

['Асфальтированная дорога до гаража. Со стороны Кировского района по Советскому шоссе первый светофор после развязки на Краснообск (6 км). Со стороны ОбьГэса третий светофор по Советскому шоссе за ОбьГэсом (3,5 км). Общественный транспорт по Советскому шоссе.']

In [ ]:
get_response("какая в поселке инфраструктура?", bert_index, index_map)

['Инфраструктура общества http://мояпланета.рф/infrastruktura']

In [ ]:
get_response("как проехать от академгородка?", bert_index, index_map)

['Координаты: 54.893991, 82.986713. Если ехать с Кировского района: по Советскому шоссе третий светофор, направо. Если ехать с Советского района: по Советскому шоссе: третий светофор налево. На шоссе имеется указатель.']

In [ ]:
get_response("как проехать от маркса?", bert_index, index_map)

['Координаты: 54.893991, 82.986713. Если ехать с Кировского района: по Советскому шоссе третий светофор, направо. Если ехать с Советского района: по Советскому шоссе: третий светофор налево. На шоссе имеется указатель.']

In [ ]:
get_response("где находится участок?", bert_index, index_map)

['Участок 30 х 40 м, 12 соток. В собственности. Категория земель – для дачного строительства, устарела. Надо переводить в категорию для индивидуального жилищного строительства. Газ, вода, канализация, электричество 3 фазы, 15 кВт на участок. Асфальтовая дорога. Участок поднят относительно дороги. Половина участка ближе к дороге супесь. Дальняя половина на более чем метр черная земля, сверху чернозем. На участке есть гараж 6 х 8 м, дом-баня 8 х 8 м с мансардой и пристроенным бетонным бассейном, автостоянка.']

In [ ]:
get_response("сколько лет участок в собственности?", bert_index, index_map)

['Участок 30 х 40 м 12 соток.']

In [ ]:
get_response("как посмотреть участок?", bert_index, index_map)

['В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.']

In [ ]:
get_response("какие документы на участок?", bert_index, index_map)

['Свидетельство о регистрации, технические условия.']

In [ ]:
index_map

{0: 'да, продаю.',
 1: 'да, продаю.',
 2: 'да, продаю.',
 3: '1 тугрик.',
 4: '1 тугрик.',
 5: '1 тугрик.',
 6: '1 тугрик.',
 7: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 8: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 9: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 10: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 11: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 12: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 13: 'Координаты: 54.893991, 82.986713. Если ехать с Кировского района: по Советскому шоссе третий светофор, направо. Если ехать с Советского района: по Советскому шоссе: третий светофор налево.На шоссе имеется указатель.',
 14: 'Координаты: 

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('index_map_planeta.p', 'wb') as fp:
    pickle.dump(index_map, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('index_map_planeta.p', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
data

{0: 'да, продаю.',
 1: 'да, продаю.',
 2: 'да, продаю.',
 3: '1 тугрик.',
 4: '1 тугрик.',
 5: '1 тугрик.',
 6: '1 тугрик.',
 7: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 8: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 9: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 10: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 11: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 12: 'В поселке пропускной режим, поэтомусамостоятельно до участка не добраться. Свяжитесь с риэлтором.',
 13: 'Координаты: 54.893991, 82.986713. Если ехать с Кировского района: по Советскому шоссе третий светофор, направо. Если ехать с Советского района: по Советскому шоссе: третий светофор налево.На шоссе имеется указатель.',
 14: 'Координаты: 

In [ ]:
morpher_qwest_tokens=set(morpher_qwest_tokens)

In [ ]:
morpher_qwest_tokens=list(morpher_qwest_tokens)

In [ ]:
morpher_qwest_tokens = ['подвал', 'академгородок', 'реквизит', 'спортивный', 'инфраструктура', 'интернет', 'цвет', 'входить', 'посёлок', 'фундамент',
     'потолочный', 'режим', 'вентиляция', 'адрес', 'работа', 'ширина', 'гараж', 'канализация', 'длина', 'стена', 'днт', 'крыша', 'охрана', 'проехать', 'предбанник',
     'разделка', 'чертёж', 'парилка', 'сайт', 'улица', 'проект', 'печь', 'продавать', 'размер', 'планета', 'телевидение', 'мег', 'банный', 'коттедж', 'расстояние',
     'площадка', 'изнутри', 'добраться', 'постройка', 'документ', 'насаждение', 'финансирование', 'карта', 'доехать', 'бухгалтер', 'член', 'потолок', 'бассейн',
     'мойка', 'комната', 'площадь', 'душевой', 'ограждение', 'труба', 'пол', 'верховодка', 'озеро', 'мечтатель', 'сотовый', 'чат', 'категория', 'водопровод',
     'количество', 'риелтор', 'риэлтор', 'тамбур', 'расписание', 'водоём', 'верховодный', 'посмотреть', 'коммуникация', 'наименование', 'попасть', 'освещение', 'природа', 'растение',
     'техэтаж', 'спортплощадка', 'юридический', 'высота', 'транспорт', 'печный', 'вентиляционный', 'геология', 'связь', 'общественный', 'зонирование', 'расположение',
     'решить', 'участок', 'стоимость', 'дом', 'окно', 'отделение', 'моечный', 'противопожарный', 'правление', 'дренаж', 'отопление', 'крыльцо', 'дверь',
     'дача', 'туалет', 'пристройка', 'устать', 'электричество', 'инженерный', 'перекрытие', 'инженер', 'детский', 'магазин', 'отмостка', 'водоснабжение', 'электрика',
     'размещение', 'электроэнергия', 'земля', 'маркс', 'супер', 'месторасположение', 'общество', 'председатель', 'связаться', 'шоссе', 'усадьба', 'вода',
     'баня', 'правило', 'проветривание', 'сантехника', 'прихожая', 'схема', 'веранда', 'показать', 'мансарда']